In [1]:
# 텐서보드를 이용하기 위해 각종 변수들을 설정하고 저장하는 방법을 익혀봅니다.

import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


In [3]:
#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope 으로 묶은 블럭은 텐서보드에서 한 레이어안에 표현해줍니다
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    # tf.summary.scalar 를 이용해 수집하고 싶은 값들을 지정할 수 있습니다.
    tf.summary.scalar('cost', cost)


In [6]:
#########
# 신경망 모델 학습
######
sess = tf.Session()


sess.run(tf.global_variables_initializer())

In [7]:
# 텐서보드에서 표시해주기 위한 텐서들을 수집합니다.
merged = tf.summary.merge_all()
# 저장할 그래프와 텐서값들을 저장할 디렉토리를 설정합니다.
writer = tf.summary.FileWriter('./logs', sess.graph)
# 이렇게 저장한 로그는, 학습 후 다음의 명령어를 이용해 웹서버를 실행시킨 뒤
# tensorboard --logdir=./logs
# 다음 주소와 웹브라우저를 이용해 텐서보드에서 확인할 수 있습니다.
# http://localhost:6006

In [8]:
# 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    # 적절한 시점에 저장할 값들을 수집하고 저장합니다.
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))


Step: 1,  Cost: 1.050
Step: 2,  Cost: 0.955
Step: 3,  Cost: 0.885
Step: 4,  Cost: 0.836
Step: 5,  Cost: 0.802
Step: 6,  Cost: 0.780
Step: 7,  Cost: 0.764
Step: 8,  Cost: 0.752
Step: 9,  Cost: 0.743
Step: 10,  Cost: 0.736
Step: 11,  Cost: 0.731
Step: 12,  Cost: 0.726
Step: 13,  Cost: 0.722
Step: 14,  Cost: 0.719
Step: 15,  Cost: 0.716
Step: 16,  Cost: 0.713
Step: 17,  Cost: 0.709
Step: 18,  Cost: 0.706
Step: 19,  Cost: 0.703
Step: 20,  Cost: 0.699
Step: 21,  Cost: 0.695
Step: 22,  Cost: 0.691
Step: 23,  Cost: 0.687
Step: 24,  Cost: 0.683
Step: 25,  Cost: 0.679
Step: 26,  Cost: 0.674
Step: 27,  Cost: 0.670
Step: 28,  Cost: 0.665
Step: 29,  Cost: 0.660
Step: 30,  Cost: 0.655
Step: 31,  Cost: 0.651
Step: 32,  Cost: 0.646
Step: 33,  Cost: 0.641
Step: 34,  Cost: 0.636
Step: 35,  Cost: 0.631
Step: 36,  Cost: 0.626
Step: 37,  Cost: 0.622
Step: 38,  Cost: 0.617
Step: 39,  Cost: 0.613
Step: 40,  Cost: 0.609
Step: 41,  Cost: 0.605
Step: 42,  Cost: 0.601
Step: 43,  Cost: 0.597
Step: 44,  Cost: 0.5

In [9]:
#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))


예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
